#Fine-tuning T5 from the Huggingface Library Simple Transformers

This notebook trains on pre-2014 Yelp data and tests on post-2014 Yelp data.

In [ ]:
!pip install simpletransformers
import pandas as pd
from simpletransformers.t5 import T5Model
from pprint import pprint
import logging
# Making sure the environment is set up correctly for anyone running this notebook
import datetime
import json
import os
import pprint
import random
import string
import sys
import tensorflow as tf
import datetime as datetime
import numpy as np
from sklearn.model_selection import StratifiedShuffleSplit
import sklearn
from google.colab import drive
from tensorflow import keras
import re

In [ ]:
# Settings
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

**Getting the Data**

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

**Load data**

In [ ]:
yelp_old_test= pd.read_pickle('/content/drive/My Drive/6862_FakeReviewDetection/data/raw/yelp_old_test.pkl')
finetune_new=pd.read_pickle('/content/drive/My Drive/6862_FakeReviewDetection/data/raw/yelp_finetune_new.pkl')
yelp_new=pd.read_pickle('/content/drive/My Drive/6862_FakeReviewDetection/data/raw/yelp_new.pkl')

**Train the model**

In [ ]:
# Model arguments. Be careful to set arguments that allow the use of custom metrics.  
# https://simpletransformers.ai/docs/t5-model/
# We use convention-based defaults for the training batch size, and the number of epochs
# Using a small # of epochs has a regularization effect (i.e. balances wanting to fit the training data well, and finding estimates
# for parameters that generalize well 

model_args = {
    "reprocess_input_data": True,
    "overwrite_output_dir": True,
    "max_seq_length": 320, 
    "train_batch_size": 32, #convention is 32; small batch sizes are noisy and offer a regularizing effect; the # of observations in the training dataset must be divisible by the train_batch_size
    "num_train_epochs": 3,
    "save_eval_checkpoints": True, 
    "save_model_every_epoch": True,
    # "silent": True,
    "evaluate_generated_text": True,
    "evaluate_during_training": True, 
    "evaluate_during_training_verbose": True, 
}


In [ ]:
# Custom metrics function

def metrics_fn(l, p):

    # Change into integer types 
    l_int = np.array(l).astype(int)
    p_int = np.array(p).astype(int)

    eval_accuracy=sklearn.metrics.accuracy_score(l, p)
    f1_score =sklearn.metrics.f1_score(l, p, labels=['0', '1'], pos_label = '1')
    auc = sklearn.metrics.roc_auc_score(l_int, p_int)
    precision = sklearn.metrics.precision_score(l, p, pos_label = '1')
    recall = sklearn.metrics.recall_score(l, p, labels=['0', '1'], pos_label = '1')
    cm = sklearn.metrics.confusion_matrix(l, p, labels=['0', '1'])
    return {'accuracy': eval_accuracy,
            'f1_score': f1_score,
            'precision': precision,
            'recall': recall,
            'auc': auc,
            'confusion matrix': cm}
    #return sum([1 if label == pred else 0 for label, pred in zip(labels, preds)])

In [ ]:

# Instantiate the model #Hard to get GPU on colab

model = T5Model("t5","t5-small", args=model_args, use_cuda = True)
# Train the model
# Common errors returned:
##                         mmap: cannnot allocate memory. In this case, "Restart runtime", and "Run all".
model.train_model(yelp_old_train, output_dir = "/content/drive/My Drive/6862_FakeReviewDetection/bestmodel_ts", eval_data=yelp_old_test, metrics = metrics_fn)


**Evaluate**

In [ ]:
# Load model
# model = T5Model("t5", "/content/drive/My Drive/6862_FakeReviewDetection/bestmodel_ts/checkpoint-852-epoch-3")

In [ ]:
# Print out model metrics on training data
'''
result_yelp_train = model.eval_model(yelp_old_train, metrics = metrics_fn)
out=pd.DataFrame([result_yelp_train['metrics']])
out.to_csv("/content/drive/My Drive/6862_FakeReviewDetection/bestmodel_ts/m_training.csv")
print(out)
'''

In [ ]:
# Print out model metrics on testing data

result_yelp_test = model.eval_model(finetune_new, metrics = metrics_fn)
out = pd.DataFrame([result_yelp_test['metrics']])
out.to_csv("/content/drive/My Drive/6862_FakeReviewDetection/bestmodel_ts/m_testing.csv")
print(out)

In [ ]:
# Print out model metrics on new data
result_yelp_new = model.eval_model(yelp_new, metrics = metrics_fn)
out=pd.DataFrame([result_yelp_new['metrics']])
out.to_csv("/content/drive/My Drive/6862_FakeReviewDetection/bestmodel_ts/m_new.csv")
print(out)